In [ ]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr

In [ ]:
pip install pytesseract

In [ ]:
pip install pillow

In [ ]:
pip install dateparser

In [5]:
pip install deskew

In [6]:
import pytesseract
import math
from typing import Tuple, Union
from deskew import determine_skew
import cv2
import dateparser
from dateparser.search import search_dates
import matplotlib.pyplot as plt
import numpy as np
import os

In [37]:
name = "25AUG2015_orgi"

In [38]:
os.mkdir(name)

In [39]:
dir = name + ".jpg"

# Rotates the image so texts are aligned


In [40]:
def rotate(
        image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]
) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2
    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background)

image = cv2.imread(dir)
grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
angle = determine_skew(grayscale)
rotated = rotate(image, angle, (0, 0, 0))
cv2.imwrite(name + '/' + 'deskew.png', rotated)
img = rotated

# Binarize Image

In [41]:
im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, im_Binarize = cv2.threshold(im_gray, 128, 192, cv2.THRESH_OTSU)
cv2.imwrite(name + '/' + 'binarized.png', im_Binarize)
img = im_Binarize

# Morphological Transformations

Erossion

In [42]:
kernel = np.ones((5,5),np.uint8)

In [43]:
erosion = cv2.erode(img,kernel,iterations = 1)
cv2.imwrite(name + '/' + 'erosion.png', erosion)

True

Dilation

In [44]:
dilation = cv2.dilate(img,kernel,iterations = 1)
cv2.imwrite(name + '/' + 'dilation.png', dilation)

True

Opening/ Errosion-Dilation

In [45]:
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
cv2.imwrite(name + '/' + 'opening.png', opening)

True

Closing/ Dilation-Erosion

In [46]:
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
cv2.imwrite(name + '/' + 'closing.png', closing)

True

Selected transformation

In [59]:
img = opening

# Adding Borders

In [48]:
color = [200, 52, 152] # 'cause purple!

# border widths; I set them all to 150
top, bottom, left, right = [15]*4

img_with_border = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

In [49]:
cv2.imwrite(name + '/' + 'borders.png', img_with_border)

True

In [50]:
img = img_with_border

# Date Extraction

In [51]:
cv2.imwrite(name + '/' + 'final.png', img)

True

In [60]:
string = pytesseract.image_to_string(img)
print(string)

) Fr ives.

BETTER IF USED BY

   

25AUG2015 CR1329T2



In [61]:
search_dates(string)

[('Fr', datetime.datetime(2021, 5, 7, 0, 0)),
 ('25AUG2015', datetime.datetime(2015, 8, 25, 0, 0))]

In [54]:
# Desired Output
search_dates('Expire: 15APR2016 bla bla bla')

[('15APR2016', datetime.datetime(2016, 4, 15, 0, 0))]

In [55]:
!zip -r /content/25AUG2015.zip /content/25AUG2015/

	zip warning: name not matched: /content/25AUG2015/

zip error: Nothing to do! (try: zip -r /content/25AUG2015.zip . -i /content/25AUG2015/)


# Pre-Modified Extraction

In [56]:
name = '25AUG2015'

In [57]:
preImg = cv2.imread(name + '/' + 'final.png')

In [58]:
string = pytesseract.image_to_string(preImg)
print(string)
li = search_dates(string)

TypeError: ignored